In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("white")


import spartan.utils.utils as spartan_utils
from imitation_agent.dataset.function_factory import ObservationFunctionFactory, ActionFunctionFactory
from imitation_agent.dataset.imitation_episode_sequence_dataset import ImitationEpisodeSequenceDataset
from imitation_agent.dataset.imitation_episode_dataset import ImitationEpisodeDataset
from imitation_agent.training import train_utils

spartan_source_dir = spartan_utils.getSpartanSourceDir()
logs_config_yaml = os.path.join(spartan_source_dir, "modules/imitation_agent/config/task/move_to_box_se2_box_in_frame.yaml")
logs_config = spartan_utils.getDictFromYamlFilename(logs_config_yaml)

imitation_src_dir = os.path.join(spartan_source_dir, "modules/imitation_agent")
data_dir = spartan_utils.get_data_dir()
logs_dir_path = os.path.join(data_dir, "pdc/imitation/move_to_box_se2")

config_yaml = os.path.join(imitation_src_dir, "config", "model", "mlp_stateless_position.yaml")
config = spartan_utils.getDictFromYamlFilename(config_yaml)


def get_dataset(num_logs):
    NUM_LOGS = num_logs
    logs_config_downsampled = train_utils.deterministic_downsample(logs_config, NUM_LOGS)
    NUM_LOGS=str(NUM_LOGS)

    obs_function = ObservationFunctionFactory.get_function(config)
    action_function = ActionFunctionFactory.action_from_config(config)
    dataset = ImitationEpisodeDataset(logs_dir_path, logs_config_downsampled, config,
                                      action_function=action_function,
                                      observation_function=obs_function)
    return dataset

In [ ]:
for num_logs in [200,100,50,30]:
    
    dataset = get_dataset(num_logs)
    NUM_LOGS = str(num_logs)
    
    from matplotlib  import cm
    import matplotlib

    def get_min_max_rewards(df):
        first = True
        for index, row in df.iterrows():
            if first:
                min_r = row["reward"]
                max_r = row["reward"]
                first = False
                continue
            if row["reward"] > max_r:
                max_r = row["reward"]
            if row["reward"] < min_r:
                min_r = row["reward"]
        return min_r, max_r

    def get_normed_reward(reward, min_r, max_r):
        return (reward - min_r)/(max_r-min_r)

    from spartan.utils import transformations
    from imitation_agent.utils.visibility_utils import check_sugar_box_in_frame


    def prune_out_of_FOV(df, dataset):
        # these don't move
        episode = dataset.episodes[dataset.episodes.keys()[0]]
        camera_num = 0
        T_W_camera = episode.get_camera_pose_matrix(camera_num)
        K = episode.get_K_matrix(camera_num)

        in_frame_list = []
        for index, row in df.iterrows():
            object_position = row["object_position"].strip("[").strip("]").split(",")
            object_position = [float(x) for x in object_position]
            object_position = np.asarray(object_position)

            object_rpy = row["object_rpy"].strip("[").strip("]").split(",")
            object_rpy = [float(x) for x in object_rpy]

            static_rotation_only = transformations.euler_matrix(0.0, 0.0, object_rpy[2], axes='sxyz')
            static_rotation_only[0,3] = object_position[0]
            static_rotation_only[1,3] = object_position[1]
            static_rotation_only[2,3] = 2.00476981e-02
            T_W_B = static_rotation_only
            in_frame = check_sugar_box_in_frame(K, T_W_camera=T_W_camera, T_W_B=T_W_B, urange=[0,640], vrange=[0,480])

            if not in_frame:
                in_frame_list.append(False)
            else:
                in_frame_list.append(True)

        df["in_frame"] = in_frame_list
        #print len(df)
        #print len(df[df["in_frame"] == True])
        return df[df["in_frame"] == True]

    def adjust_reward(df):
        df['reward'] += 1.2
        df['reward'] = np.minimum(0.0,df['reward'])
        return df

    def create_distribution_plots(ax, df, dataset, min_r, max_r, first, use_yaw, ax_labels):

        xs = []
        ys = []
        cs = []

        episode = dataset.episodes[dataset.episodes.keys()[0]]
        camera_num = 0
        T_W_camera = episode.get_camera_pose_matrix(camera_num)
        K = episode.get_K_matrix(camera_num)

        # plot heatmap
        for index, row in df.iterrows():
            this_index = row["index"]

            object_position = row["object_position"].strip("[").strip("]").split(",")
            object_position = [float(x) for x in object_position]

            object_rpy = row["object_rpy"].strip("[").strip("]").split(",")
            object_rpy = [float(x) for x in object_rpy]

            object_position = np.asarray(object_position + object_rpy)

            #jet = cm.get_cmap("jet")
            #jet = cm.get_cmap("coolwarm")
            #normed_reward = (-1.0*get_normed_reward(reward, min_r, max_r))+1.0
            #normed_reward = get_normed_reward(reward, min_r, max_r)
            if use_yaw:
                x = object_position[5]
            else:
                x = object_position[0]
            y = object_position[1]
            xs.append(x)
            ys.append(y)
            #cs.append(jet(normed_reward))
            cs.append(row["reward"])
        [xs, ys, cs] = [np.asarray(x) for x in [xs,ys,cs]]

        hm = ax.scatter(xs, ys, c=cs, vmin=-2.0, vmax=0, cmap = cm.coolwarm_r)
        #hm = ax.scatter(ys, xs, c=cs, cmap = cm.coolwarm_r)
        #plt.colorbar(hm)

        # plot training examples
        
        yaw_train = []
        y_train = []
        for log_name in dataset.episodes.keys():

            episode = dataset.episodes[log_name]

            x = episode.get_entry(0)["observations"]["object_pose_cheat_data"]["position"]["x"]
            y = episode.get_entry(0)["observations"]["object_pose_cheat_data"]["position"]["y"]
            
            T_W_B = spartan_utils.homogenous_transform_from_dict(episode.get_entry(0)['observations']["object_pose_cheat_data"])
            angle_axis = spartan_utils.angle_axis_from_rotation_matrix(T_W_B[:3,:3])
            yaw = angle_axis[2]

            if use_yaw:
                x = yaw
                
            yaw_train.append(yaw)
            y_train.append(y)


            sc = ax.scatter(x, y, c="gray", alpha=0.4, marker="x")
            
        yaw_train = np.asarray(yaw_train)
        y_train = np.asarray(y_train)
        from scipy.spatial import ConvexHull
        points = np.vstack((yaw_train,y_train)).transpose()
        
        hull = ConvexHull(points)
        for simplex in hull.simplices:
            ax.plot(points[simplex, 0], points[simplex, 1], ls='--', color="gray")

        #ax.axis('equal')
        #ax.set(ylim=(-0.2, 0.2), xlim=(0.6, 0.7))
        ax.set_title(row['name'])
        if not first:
            #ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.set_xlabel(ax_labels[0])
            ax.set_ylabel(ax_labels[1])
        return hm


    dfs = []  


    folders = []
    for folder in sorted(os.listdir(os.path.join(spartan_source_dir, "sandbox/experiment_02/logs-"+NUM_LOGS))):
        #print folder
        if "bak" in folder or folder == "05-endtoend" or folder=="01-gt-pose":
            continue
        folders.append(folder)
        path_to_move_to_box_folder = os.path.join(spartan_source_dir, "sandbox","experiment_02/logs-"+NUM_LOGS,folder)
        path_to_csv = os.path.join(path_to_move_to_box_folder, "results.csv")
        df = pd.read_csv(path_to_csv, index_col=0)
        df["name"] = folder[3:]
        dfs.append(df)

    for i, df in enumerate(dfs):
        dfs[i] = prune_out_of_FOV(df, dataset)

    for i, df in enumerate(dfs):
        dfs[i] = adjust_reward(df)    

    min_r = 1e6
    max_r = -1e6
    for df in dfs:
        this_min, this_max = get_min_max_rewards(df)
        if this_min < min_r:
            min_r = this_min
        if this_max > max_r:
            max_r = this_max

    # SETTING THIS MANUALLY FOR SCALE
    #min_r = -20
    min_r = -5

    # fig, axes = plt.subplots(ncols=len(dfs),nrows=1)
    # fig.set_figheight(5)
    # fig.set_figwidth(3* len(dfs))
    # for i, df in enumerate(dfs):
    #     if len(dfs) == 1:
    #         ax = axes
    #     else:
    #         ax = axes[i]
    #     if i == 0:
    #         first = True
    #     else:
    #         first = False
    #     ax_labels = ("x, box position", "y, box position")
    #     hm = create_distribution_plots(ax, df, dataset, min, max, first=first, use_yaw=False, ax_labels=ax_labels)
    # #plt.colorbar(hm, pad=0.2)

    # cbaxes = fig.add_axes([0.92, 0.12, 0.02, 0.76]) 
    # cb = plt.colorbar(hm, cax = cbaxes) 
    # cb.set_label('Task success metric (higher is better)', labelpad=20, rotation=270)
    # plt.show()

    fig, axes = plt.subplots(ncols=len(dfs),nrows=1)
    fig.set_figheight(5)
    fig.set_figwidth(3* len(dfs))
    for i, df in enumerate(dfs):
        if len(dfs) == 1:
            ax = axes
        else:
            ax = axes[i]
        if i == 0:
            first = True
        else:
            first = False
        ax_labels = ("theta, box position", "y, box position")
        hm = create_distribution_plots(ax, df, dataset, min, max, first=first, use_yaw=True, ax_labels=ax_labels)
    #plt.colorbar(hm, pad=0.2)

    cbaxes = fig.add_axes([0.92, 0.12, 0.02, 0.76]) 
    cb = plt.colorbar(hm, cax = cbaxes) 
    cb.set_label('Task success metric (higher is better)', labelpad=20, rotation=270)
    
    fig.savefig("MTBSE2"+NUM_LOGS+".pdf")

    plt.show()

In [ ]:
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import matplotlib as mpl
import matplotlib.pyplot as plt

sns.set_style("white", {"axes.grid": True})

def plot_tensorflow_log(axis_pair, path, color, label, use_yaxis_label):

    event_acc = EventAccumulator(path)#, tf_size_guidance)
    event_acc.Reload()

    # Show all tags in the log file
    #print(event_acc.Tags())

    training_accuracies =   event_acc.Scalars('loss')
    validation_accuracies = event_acc.Scalars('test_loss_average')

    x_train_steps = []
    x_train_values = []
    
    
    for i in xrange(len(training_accuracies)):
        x_train_steps.append(training_accuracies[i][1]) # 1 is step
        x_train_values.append(training_accuracies[i][2]) # 2 is step
    
#     from scipy.signal import savgol_filter
#     x_train_values_filt = savgol_filter(x_train_values, 21, 2)

    #from scipy.signal import medfilt
    #x_train_values_filt = medfilt(x_train_values, 21)
    
    w = 0.90
    x_train_values = np.asarray(x_train_values)
    x_train_values_filt = x_train_values*0.0
    x_train_values_filt[0] = x_train_values[0]
    for i, v in enumerate(x_train_values[1:]):
        x_train_values_filt[i+1] = x_train_values_filt[i]*w + (1-w)*x_train_values[i]
   
    
#     from scipy import signal
#     xn = x_train_values
#     b, a = signal.butter(3, 0.05)
#     zi = signal.lfilter_zi(b, a)
#     z, _ = signal.lfilter(b, a, xn, zi=zi*xn[0])
#     z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
#     y = signal.filtfilt(b, a, xn)
#     x_train_values_filt = y

        
    x_validation_steps = []
    x_validation_values = []
    
    for i in xrange(len(validation_accuracies)):
        x_validation_steps.append(validation_accuracies[i][1])
        x_validation_values.append(validation_accuracies[i][2])
        

    axis_pair[0].plot(np.asarray(x_train_steps), np.asarray(x_train_values_filt), color=color, label=label, alpha=1.0)
    axis_pair[0].plot(np.asarray(x_train_steps), np.asarray(x_train_values), color=color, alpha=0.1)
    axis_pair[1].plot(np.asarray(x_validation_steps), np.asarray(x_validation_values), label=label, color=color)

    #axis_pair[0].set_xlabel("Steps")
    axis_pair[1].set_xlabel("Steps")
    if use_yaxis_label:
        axis_pair[0].set_ylabel("Loss")
        axis_pair[1].set_ylabel("Loss")
    axis_pair[0].set_xticks([])
    #axis_pair[0].set_title("Train")
    axis_pair[0].set(ylim=(0.5e-4, 1.5e-4))
    axis_pair[1].set(ylim=(0.5e-4, 1.5e-4))
    #axis_pair[1].set_title("Validation")


def get_tf_events_file(path):
    return sorted(os.listdir(path))[-1]
    
fig, axes = plt.subplots(nrows=2,ncols=4)
fig.set_figheight(5)
fig.set_figwidth(15)



#colors = ["black", "red", "orange", "green", "blue", "purple"]
colors = ["red", "orange", "blue", "green", "purple"]

#folders.insert(0,"00-blind")

def folder_to_plot_name(name):
    if name == "gt-3D-points":
        return "Ground truth 3D points"
    if name == "gt-3D-points-projected":
        return "Ground truth 2D image coordinates"
    if name == "endtoend-fullwidth":
        return "End-to-End"
    else:
        return "Dense Descriptors"


for num_index, NUM_LOGS in enumerate(["30", "50", "100", "200"]):
    base = os.path.join(data_dir, "pdc/imitation/trained_models/mlp_position/experiment_02-aug12-logs-"+NUM_LOGS)
    folders = []
    for folder in sorted(os.listdir(os.path.join(spartan_source_dir, "sandbox/experiment_02/logs-"+NUM_LOGS))):
        #print folder
        if "bak" in folder or folder == "05-endtoend" or folder=="01-gt-pose":
            continue
        folders.append(folder)

    counter = 0
    for color, folder in zip(colors, folders):
        counter+=1
        path_to = os.path.join(base,folder,"tensorboard")
        tf_file = get_tf_events_file(path_to)
        tf_file_full = os.path.join(path_to, tf_file)
        print tf_file_full
        axis_pair = [axes[0, num_index], axes[1,num_index]]
        use_yaxis_label = (num_index==0)
        plot_tensorflow_log(axis_pair, tf_file_full, color, label=folder_to_plot_name(folder[3:]), use_yaxis_label=use_yaxis_label)
    #     if counter == 2:
    #         break

    axes[0,num_index].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    axes[1,num_index].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

#lgd = axes[0,3].legend(loc='upper right', bbox_to_anchor=(-2.4,3.0),frameon=True)
lgd = axes[0,3].legend(loc='upper right', bbox_to_anchor=(2.4,1.05), frameon=True)

fig.savefig("MTBSE2-samplecomplexity-traintest2.pdf", bbox_inches="tight")
#fig.savefig('MTBSE2-samplecomplexity-traintest2.pdf', bbox_extra_artists=(lgd), bbox_inches='tight')
plt.show()


In [ ]:
print "Training times"
print "---------------"
import datetime

for folder in folders:
    path_to = os.path.join(base,folder,"tensorboard")
    tf_file = get_tf_events_file(path_to)
    tf_file_full = os.path.join(path_to, tf_file)
    event_acc = EventAccumulator(tf_file_full)#, tf_size_guidance)
    event_acc.Reload()
    training_accuracies =   event_acc.Scalars('loss')
    seconds = training_accuracies[-1][0] - training_accuracies[0][0]
    #print str(datetime.timedelta(seconds=seconds)) 
    #print '%d hr, %d minutes, %02d seconds' % (seconds / 60 / 60,  seconds/ 60 % 60, seconds % 60)
    print '{:25}'.format(folder[3:]),  '%d hr, %d minutes' % (seconds / 60 / 60,  seconds/ 60 % 60)
    
    


In [ ]:
print folders

In [ ]:
def create_time_drift_plot(dfs):
    for df in dfs:
        plt.scatter(df["index"], df["termination_time"])
    plt.show()
    
    plt.scatter(dfs[-1]["index"], df["termination_time"])
    plt.show()
    
create_time_drift_plot(dfs)

In [ ]:
import matplotlib

def create_violin_plot(dfs):
    df_all = pd.concat(dfs)
    sns.set(style="whitegrid")
    ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
create_violin_plot(dfs)

In [ ]:
# great example: https://seaborn.pydata.org/examples/distplot_options.html
# full documentation: https://seaborn.pydata.org/generated/seaborn.distplot.html
def create_dist_plot(dfs):
    df_all = pd.concat(dfs)
    
    fig, axes = plt.subplots(1, len(dfs), figsize=(9, 3), sharey=True)
    
    
    #axes[0].set_ylim([-20,0])
    sns.set(style="whitegrid")
    for i, df in enumerate(dfs):
#         if not isinstance(axes, list):
#             this_ax = axes
#         else:
#             this_ax = axes[i]
        this_ax = axes[i]
        sns.distplot(df["reward"], vertical=True, ax=this_ax, bins=20)
        #ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
        #fig = matplotlib.pyplot.gcf()
        #fig.set_size_inches(18.5, 10.5)
    plt.show()

    fig, axes = plt.subplots(1, len(dfs), figsize=(9, 3), sharey=True)
#     if not isinstance(axes, list):
#         axes.set_ylim([-2,-1])
#     else:
#         axes[0].set_ylim([-2,-1])
    
    sns.set(style="whitegrid")
    for i, df in enumerate(dfs):
#         if not isinstance(axes, list):
#             this_ax = axes
#         else:
        this_ax = axes[i]
        sns.distplot(df["reward"], vertical=True, ax=this_ax, bins=20, rug=True)
        #ax = sns.violinplot(x=df_all["name"], y=df_all["reward"])
        #fig = matplotlib.pyplot.gcf()
        #fig.set_size_inches(18.5, 10.5)
    plt.show()

    
    
create_dist_plot(dfs)